Convert Lasagne's [VGG16 weights](https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg16.pkl) into a smaller file.

Original source: https://gist.github.com/ksimonyan/211839e770f7b538e2d8
License: see http://www.robots.ox.ac.uk/~vgg/research/very_deep/

Based on code in the [Lasagne Recipes](https://github.com/Lasagne/Recipes) repository, and code described in [this](https://github.com/dnouri/nolearn/issues/163#issuecomment-201696232) issue.

In [37]:
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.nonlinearities import softmax


def build_model():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    net['conv1_1'] = ConvLayer(
        net['input'], 64, 3, pad=1, flip_filters=False)
    net['conv1_2'] = ConvLayer(
        net['conv1_1'], 64, 3, pad=1, flip_filters=False)
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(
        net['pool1'], 128, 3, pad=1, flip_filters=False)
    net['conv2_2'] = ConvLayer(
        net['conv2_1'], 128, 3, pad=1, flip_filters=False)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(
        net['pool2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_2'] = ConvLayer(
        net['conv3_1'], 256, 3, pad=1, flip_filters=False)
    net['conv3_3'] = ConvLayer(
        net['conv3_2'], 256, 3, pad=1, flip_filters=False)
    net['pool3'] = PoolLayer(net['conv3_3'], 2)
    net['conv4_1'] = ConvLayer(
        net['pool3'], 512, 3, pad=1, flip_filters=False)
    net['conv4_2'] = ConvLayer(
        net['conv4_1'], 512, 3, pad=1, flip_filters=False)
    net['conv4_3'] = ConvLayer(
        net['conv4_2'], 512, 3, pad=1, flip_filters=False)
    net['pool4'] = PoolLayer(net['conv4_3'], 2)
    net['conv5_1'] = ConvLayer(
        net['pool4'], 512, 3, pad=1, flip_filters=False)
    net['conv5_2'] = ConvLayer(
        net['conv5_1'], 512, 3, pad=1, flip_filters=False)
    net['conv5_3'] = ConvLayer(
        net['conv5_2'], 512, 3, pad=1, flip_filters=False)
    net['pool5'] = PoolLayer(net['conv5_3'], 2)
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc6_dropout'] = DropoutLayer(net['fc6'], p=0.5)
    net['fc7'] = DenseLayer(net['fc6_dropout'], num_units=4096)
    net['fc7_dropout'] = DropoutLayer(net['fc7'], p=0.5)
    net['fc8'] = DenseLayer(
        net['fc7_dropout'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    return net

In [38]:
from lasagne.layers import set_all_param_values
import pickle

with open('../data/vgg16.pkl', 'rb') as f:
    params = pickle.load(f, encoding='latin-1')

print(params.keys())

dict_keys(['mean value', 'synset words', 'param values', 'model name'])


In [39]:
net = build_model()

In [40]:
output_layer = net['prob']
set_all_param_values(output_layer, params['param values'])

In [41]:
import numpy as np
from lasagne.layers import get_all_param_values
np.savez('../data/vgg16_loss_net.npz', *get_all_param_values(net['conv5_3']))

In [42]:
# Not certain that bzip-ing a numpy zipped file is the best way of handling this, but it works great
import bz2
from shutil import copyfileobj

with open('../data/vgg16_loss_net.npz', 'rb') as input:
    with bz2.BZ2File('../data/vgg16_loss_net.npz.bz2', 'wb', compresslevel=9) as output:
        copyfileobj(input, output)